In [21]:
import networkx as nx
import numpy as np
import pandas as pd
from cartoframes.viz import *
import torch
import torch.nn.functional as F
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv, GATConv
from sklearn.preprocessing import StandardScaler
from tqdm import tqdm
import pickle
import random

In [22]:
# 각 열별 정규화
def standardize_dataframe(df, exclude_columns=[]):
    scaler = StandardScaler()
    standardized_df = df.copy()
    for column in standardized_df.columns:
        if column not in exclude_columns:
            standardized_df[column] = scaler.fit_transform(standardized_df[[column]])
    return standardized_df

In [23]:
# 도로 임베딩 불러오기
emb = pd.read_csv('../../Dataset/Raw_Embeddings/Road_Embeddings.csv')
# 정규화
emb = standardize_dataframe(emb, ['ADM_NM'])
features = torch.tensor(emb.iloc[:, 1:].values, dtype=torch.float32)   # 첫 번째 열 제외 (ADM_NM)
emb

,ADM_NM,road_count,total_length,tunnel_count_general,tunnel_count_building_passage,bridge_count_general,bridge_count_viaduct,highway_count_residential,highway_count_primary,highway_count_tertiary,...,highway_count_secondary,highway_count_secondary_link,highway_count_unclassified,highway_count_trunk,highway_count_trunk_link,highway_count_motorway_link,highway_count_crossing,highway_count_road,highway_count_motorway,area
0,사직동,-0.692377,-0.409433,-0.357050,-0.111429,-0.435506,-0.123797,-0.942891,1.894501,0.272126,...,-1.021656,-0.561194,-0.269343,-0.332476,-0.344662,-0.136229,-0.048622,-0.068843,-0.075449,-0.170988
1,삼청동,-1.367302,-1.502476,-0.357050,-0.111429,-0.435506,-0.123797,-1.232211,-0.706835,-0.606691,...,-1.021656,-0.561194,-0.269343,-0.332476,-0.344662,-0.136229,-0.048622,-0.068843,-0.075449,0.030718
2,부암동,-0.140790,0.423654,-0.357050,-0.111429,2.631115,-0.123797,-0.128938,-0.101873,0.198891,...,-0.308550,-0.376453,-0.269343,-0.332476,-0.344662,-0.136229,-0.048622,-0.068843,-0.075449,0.535565
3,평창동,1.030906,2.708098,-0.357050,-0.111429,0.440672,-0.123797,1.336948,1.410532,-0.826395,...,-1.021656,0.362510,-0.269343,-0.332476,-0.344662,-0.136229,-0.048622,-0.068843,-0.075449,4.759278
4,한남동,2.480109,3.030663,0.552194,-0.111429,0.659716,-0.123797,2.405502,1.229043,0.784768,...,1.117662,5.165770,0.079897,0.216045,-0.344662,-0.136229,-0.048622,-0.068843,-0.075449,1.014827
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
419,가회동,-1.278225,-1.450547,-0.357050,-0.111429,-0.435506,-0.123797,-1.185920,-0.827828,-0.020813,...,-1.021656,-0.561194,-0.269343,-0.332476,-0.344662,-0.136229,-0.048622,-0.068843,-0.075449,-0.562516
420,종로1·2·3·4가동,1.191929,1.565709,6.916902,-0.111429,-0.435506,-0.123797,0.337831,5.826754,1.041090,...,1.919906,1.101473,-0.269343,-0.332476,-0.344662,-0.136229,-0.048622,-0.068843,-0.075449,0.625696
421,방배2동,0.527282,0.634357,0.097572,-0.111429,-0.435506,-0.123797,0.002220,-0.222866,1.260794,...,-0.367976,0.547251,1.476856,0.764565,0.265087,-0.136229,-0.048622,-0.068843,-0.075449,0.300902
422,미아동,1.918243,0.850983,-0.357050,-0.111429,-0.435506,-0.123797,2.347638,0.503089,-0.606691,...,-0.813667,-0.561194,-0.269343,-0.332476,-0.344662,-0.136229,-0.048622,-0.068843,-0.075449,-0.371862


In [24]:
# 도로 그래프 확인
with open('../../Dataset/Road_Graph/road_graph.gpickle', 'rb') as f:
    G1 = pickle.load(f)

edges = G1.edges(data=True)   # 간선정보 추출
node_to_idx = {name: idx for idx, name in enumerate(emb['ADM_NM'])}  # 동이름 -> 인덱스 번호 변환
edge_index = torch.tensor([[node_to_idx[edge[0]], node_to_idx[edge[1]]] for edge in edges], dtype=torch.long).T   # edge_index 확인

# 가중치 로드
edge_weight = torch.tensor([edge[2]['weight'] for edge in edges], dtype=torch.float32)

In [25]:
# PyTorch Geometric 데이터 객체 생성
data = Data(x=features, edge_index=edge_index, edge_weight=edge_weight)

print(data)

# Check for NaN in features
print(torch.isnan(data.x).any())  # True면 데이터에 NaN 존재
print(torch.isnan(data.edge_weight).any())  # True면 데이터에 NaN 존재

# Check for very large or very small values
print(data.x.max(), data.x.min())  # 값 범위 확인
print(data.edge_weight.max(), data.edge_weight.min())

Data(x=[424, 23], edge_index=[2, 1218], edge_weight=[1218])
tensor(False)
tensor(False)
tensor(20.5670) tensor(-1.6097)
tensor(72.) tensor(0.1000)


In [26]:
# GCN 모델 정의
class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)

    def forward(self, x, edge_index, edge_weight):
        x = self.conv1(x, edge_index, edge_weight)
        x = F.relu(x)
        x = self.conv2(x, edge_index, edge_weight)
        return x

In [27]:
# GAT 모델 정의
class GAT(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, heads=1):
        super(GAT, self).__init__()
        self.conv1 = GATConv(in_channels, hidden_channels, heads=heads)
        self.conv2 = GATConv(hidden_channels * heads, out_channels, heads=1)

    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return x

In [28]:
# Negative Sampling 구현
def negative_sampling(edge_index, num_nodes, num_neg_samples):
    """Negative samples 생성"""
    edge_set = set(tuple(edge) for edge in edge_index.T.tolist())
    neg_samples = []
    while len(neg_samples) < num_neg_samples:
        i, j = np.random.randint(0, num_nodes, size=2)
        if i != j and (i, j) not in edge_set and (j, i) not in edge_set:
            neg_samples.append((i, j))
    return torch.tensor(neg_samples, dtype=torch.long).T

In [29]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  # 멀티 GPU를 사용하는 경우
        torch.backends.cudnn.deterministic = True
        torch.backends.cudnn.benchmark = False

In [30]:
# 학습 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data = data.to(device)

model_gcn = GCN(data.num_node_features, 64, 35).to(device)
model_gat = GAT(data.num_node_features, 64, 35, heads=4).to(device)  # heads=4는 각 노드에서 4개의 attention heads 사용

# 최적화 방법 설정
optimizer_gcn = torch.optim.Adam(model_gcn.parameters(), lr=0.01, weight_decay=5e-4)
optimizer_gat = torch.optim.Adam(model_gat.parameters(), lr=0.01, weight_decay=5e-4)

In [31]:
def train_model(model, data, optimizer, num_epochs=200, patience=20, seed=42):
    """
    GCN 또는 GAT 모델 학습을 위한 함수 (Early Stopping 시 best_loss 기준으로 z 반환).
    
    Args:
        model (torch.nn.Module): 학습할 모델 (GCN 또는 GAT).
        data (torch_geometric.data.Data): 학습에 사용할 데이터.
        optimizer (torch.optim.Optimizer): 최적화 알고리즘.
        num_epochs (int): 학습할 에폭 수.
        patience (int): Early Stopping 기준.
        seed (int): 재현 가능한 결과를 위한 Seed 값.

    Returns:
        torch.Tensor: `best_loss` 기준으로 저장된 노드 임베딩.
    """
    # Seed 고정
    set_seed(seed)

    best_loss = float('inf')
    counter = 0
    best_z = None  # `best_loss` 시점의 임베딩 저장

    for epoch in tqdm(range(num_epochs), desc="Training Progress"):
        model.train()
        optimizer.zero_grad()

        # Forward pass
        if isinstance(model, GAT):  # GAT 모델에는 edge_weight 사용하지 않음
            z = model(data.x, data.edge_index)
        else:  # GCN 모델에는 edge_weight 전달
            z = model(data.x, data.edge_index, data.edge_weight)

        # Positive edges
        pos_edges = data.edge_index.T
        pos_scores = (z[pos_edges[:, 0]] * z[pos_edges[:, 1]]).sum(dim=1)

        # Negative edges
        neg_edges = negative_sampling(data.edge_index, data.num_nodes, pos_edges.size(0) // 2)
        neg_scores = (z[neg_edges[0]] * z[neg_edges[1]]).sum(dim=1)

        # Loss 계산
        pos_loss = -F.logsigmoid(pos_scores).mean()
        neg_loss = -F.logsigmoid(-neg_scores).mean()
        loss = pos_loss + neg_loss

        loss.backward()
        optimizer.step()

        # Loss 출력
        tqdm.write(f"Epoch {epoch + 1}, Loss: {loss.item()}")

        # Early Stopping 로직
        if loss.item() < best_loss:
            best_loss = loss.item()  # 최적 Loss 갱신
            best_z = z.detach().cpu()  # `best_loss` 시점의 임베딩 저장
            counter = 0  # 카운터 초기화
        else:
            counter += 1

        if counter >= patience:
            tqdm.write(f"Early stopping at epoch {epoch + 1}")
            break

    return best_z  # 최적 Loss 기준의 임베딩 반환

In [32]:
z_gcn = train_model(model_gcn, data, optimizer_gcn, num_epochs=200, patience=20)
z_gat = train_model(model_gat, data, optimizer_gat, num_epochs=200, patience=20)

Training Progress:  11%|█         | 22/200 [00:00<00:01, 103.99it/s]

Epoch 1, Loss: 10.854968070983887
Epoch 2, Loss: 4.195703506469727
Epoch 3, Loss: 3.891542673110962
Epoch 4, Loss: 2.9166908264160156
Epoch 5, Loss: 2.745389461517334
Epoch 6, Loss: 2.968179702758789
Epoch 7, Loss: 2.5635128021240234
Epoch 8, Loss: 2.194786787033081
Epoch 9, Loss: 1.765521764755249
Epoch 10, Loss: 1.7330124378204346
Epoch 11, Loss: 1.3936694860458374
Epoch 12, Loss: 1.5492310523986816
Epoch 13, Loss: 1.4066708087921143
Epoch 14, Loss: 1.2674479484558105
Epoch 15, Loss: 1.2559254169464111
Epoch 16, Loss: 1.4443254470825195
Epoch 17, Loss: 1.4217560291290283
Epoch 18, Loss: 1.2687187194824219
Epoch 19, Loss: 1.3363960981369019
Epoch 20, Loss: 1.2622857093811035
Epoch 21, Loss: 1.2414218187332153
Epoch 22, Loss: 1.2483720779418945


Training Progress:  16%|█▋        | 33/200 [00:00<00:01, 104.35it/s]

Epoch 23, Loss: 1.1925228834152222
Epoch 24, Loss: 1.221111536026001
Epoch 25, Loss: 1.1577825546264648
Epoch 26, Loss: 1.1875665187835693
Epoch 27, Loss: 1.1525604724884033
Epoch 28, Loss: 1.1516773700714111
Epoch 29, Loss: 1.1523747444152832
Epoch 30, Loss: 1.1137609481811523
Epoch 31, Loss: 1.081343650817871
Epoch 32, Loss: 1.1244688034057617
Epoch 33, Loss: 1.0863327980041504
Epoch 34, Loss: 1.1355493068695068
Epoch 35, Loss: 1.1820753812789917
Epoch 36, Loss: 1.0934250354766846
Epoch 37, Loss: 1.2151492834091187
Epoch 38, Loss: 1.1085032224655151
Epoch 39, Loss: 1.070314645767212
Epoch 40, Loss: 1.070676565170288
Epoch 41, Loss: 1.107494592666626
Epoch 42, Loss: 1.1273690462112427
Epoch 43, Loss: 1.073858380317688
Epoch 44, Loss: 1.134537696838379


Training Progress:  28%|██▊       | 55/200 [00:00<00:01, 104.03it/s]

...

Epoch 55, Loss: 0.9035993814468384
Epoch 56, Loss: 0.905413806438446
Epoch 57, Loss: 0.8720277547836304
Epoch 58, Loss: 0.873081386089325
Epoch 59, Loss: 0.8754671216011047
Epoch 60, Loss: 0.8964792490005493
Epoch 61, Loss: 0.9237464666366577
Epoch 62, Loss: 0.897783100605011
Epoch 63, Loss: 0.8769207000732422
Epoch 64, Loss: 0.8772110939025879
Epoch 65, Loss: 0.8820030093193054
Epoch 66, Loss: 0.8662734031677246
Epoch 67, Loss: 0.8803034424781799
Epoch 68, Loss: 0.8236425518989563
Epoch 69, Loss: 0.8669480085372925
Epoch 70, Loss: 0.8953091502189636


Training Progress:  36%|███▌      | 72/200 [00:00<00:01, 87.10it/s]

Epoch 71, Loss: 0.869328498840332
Epoch 72, Loss: 0.8839541673660278


Training Progress:  44%|████▎     | 87/200 [00:00<00:01, 87.07it/s]

Epoch 73, Loss: 0.8536001443862915
Epoch 74, Loss: 0.8921664953231812
Epoch 75, Loss: 0.852906346321106
Epoch 76, Loss: 0.8671475648880005
Epoch 77, Loss: 0.8624761700630188
Epoch 78, Loss: 0.9227997660636902
Epoch 79, Loss: 0.9739560484886169
Epoch 80, Loss: 0.907861590385437
Epoch 81, Loss: 0.906217098236084
Epoch 82, Loss: 0.8894060850143433
Epoch 83, Loss: 0.862169623374939
Epoch 84, Loss: 0.8739551305770874
Epoch 85, Loss: 0.8553781509399414
Epoch 86, Loss: 0.868458092212677
Epoch 87, Loss: 0.8802168965339661
Epoch 88, Loss: 0.8905534148216248
Early stopping at epoch 88


In [33]:
def save_embeddings(z, emb, output_path, model_name="model"):
    """
    학습된 임베딩을 CSV 파일로 저장하는 함수.

    Args:
        z (torch.Tensor): 학습된 임베딩 (노드 임베딩).
        emb (pd.DataFrame): 원본 데이터프레임 (ADM_NM 열 포함).
        output_path (str): 저장할 CSV 파일 경로.
        model_name (str): 임베딩 컬럼에 추가할 모델 이름 접두사 (예: "GCN", "GAT").

    Returns:
        None
    """
    # GPU 텐서를 NumPy 배열로 변환
    z_numpy = z.detach().cpu().numpy()

    # 새로운 데이터프레임 생성: ADM_NM + 학습된 임베딩
    embedding_columns = [f"{model_name}_embed{i}" for i in range(z_numpy.shape[1])]  # 임베딩 컬럼 이름 생성
    emb_updated = pd.DataFrame(emb['ADM_NM'], columns=['ADM_NM'])  # ADM_NM 열만 복사
    emb_updated[embedding_columns] = z_numpy  # 임베딩 추가

    # CSV로 저장
    # emb_updated.to_csv(output_path, index=False)
    # print(f"Embeddings saved to: {output_path}")
    
    return emb_updated

In [35]:
# # 학습된 임베딩 저장
gcn_path = "../../Dataset/Road_Graph/GCN_Road_Embeddings.csv"
gat_path = "../../Dataset/Road_Graph/GAT_Road_Embeddings.csv"

gcn_df = save_embeddings(z_gcn, emb, gcn_path, 'GCN')
gat_df = save_embeddings(z_gat, emb, gat_path, 'GAT')

In [36]:
gcn_df

,ADM_NM,GCN_embed0,GCN_embed1,GCN_embed2,GCN_embed3,GCN_embed4,GCN_embed5,GCN_embed6,GCN_embed7,GCN_embed8,...,GCN_embed25,GCN_embed26,GCN_embed27,GCN_embed28,GCN_embed29,GCN_embed30,GCN_embed31,GCN_embed32,GCN_embed33,GCN_embed34
0,사직동,-0.113967,-0.145764,0.053565,0.307150,-0.113376,0.105927,-0.214251,-0.072469,-0.068509,...,0.084458,-0.141207,0.105410,-0.160377,0.042990,0.025559,-0.034236,0.010405,0.178941,-0.258927
1,삼청동,-0.460062,-0.417919,0.294666,1.075638,-0.696440,0.528849,-0.768585,-0.127918,-0.185664,...,0.166929,-0.528783,0.248094,-0.600811,0.224019,0.217416,-0.260131,0.234869,0.561494,-0.910570
2,부암동,-0.240787,-0.246660,0.208716,0.635105,-0.364379,0.269012,-0.487198,-0.064919,-0.119107,...,0.050744,-0.267290,0.123039,-0.352817,0.135905,0.166968,-0.136537,0.191401,0.336411,-0.555364
3,평창동,-0.014308,0.143441,-0.209128,0.145030,-0.002484,-0.187231,-0.193631,-0.079847,0.002095,...,0.038792,0.046413,0.106894,0.026963,-0.004898,-0.023650,0.029114,0.039258,0.241739,0.009103
4,한남동,-0.044745,-0.002345,0.080078,0.029406,-0.149812,-0.018572,0.172200,-0.205198,0.032986,...,-0.024681,0.094311,-0.252859,-0.036507,-0.022770,0.081767,-0.006708,0.138030,0.121597,0.147482
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
419,가회동,-0.267630,0.002126,0.063739,0.451149,-0.312092,0.308443,-0.215596,0.116035,0.184929,...,0.068565,-0.366840,0.231913,-0.082770,0.107354,0.113222,-0.131414,0.043047,0.265318,-0.190545
420,종로1·2·3·4가동,-0.429459,-0.463181,0.166156,1.108531,-0.769806,0.497920,-0.687034,-0.250774,-0.214923,...,0.322816,-0.531523,0.290804,-0.683506,0.187468,0.171691,-0.250441,0.054683,0.614987,-1.012548
421,방배2동,0.294545,0.174868,-0.172202,-0.180617,-0.205243,-0.014143,0.141577,-0.072925,0.192465,...,0.173619,-0.324178,-0.060888,0.341314,0.032318,0.230805,0.064690,0.078176,0.023722,-0.288774
422,미아동,0.045713,0.143000,-0.597584,0.224257,0.738796,0.276811,0.244921,-0.163800,0.033103,...,-0.027929,0.390088,0.148383,0.461511,-0.079860,-0.099557,0.591009,0.110919,0.489388,0.226877


In [37]:
gat_df

,ADM_NM,GAT_embed0,GAT_embed1,GAT_embed2,GAT_embed3,GAT_embed4,GAT_embed5,GAT_embed6,GAT_embed7,GAT_embed8,...,GAT_embed25,GAT_embed26,GAT_embed27,GAT_embed28,GAT_embed29,GAT_embed30,GAT_embed31,GAT_embed32,GAT_embed33,GAT_embed34
0,사직동,-0.126598,-0.675009,-0.359300,0.196953,-1.170815,0.083285,-0.261959,-0.234460,0.443212,...,-0.826534,0.467861,0.447222,-0.344162,0.764721,-0.101463,-0.941549,-0.074837,-0.794512,0.904834
1,삼청동,-0.258355,-0.481924,-0.359263,-0.244259,-0.680146,0.011330,-0.109652,-0.239343,0.481987,...,-0.593243,0.245068,0.124202,0.213391,0.466053,0.001734,-0.436373,0.662552,-0.521592,0.489228
2,부암동,-0.324693,-0.514466,-0.187272,0.128022,-0.492366,-0.059508,-0.018852,-0.435758,0.441834,...,-0.179962,0.209398,-0.210486,0.309511,0.387809,0.112115,-0.148782,0.805479,-0.508751,0.436858
3,평창동,-0.255704,-0.040217,0.222770,0.405416,-0.002204,-0.093632,-0.100117,-0.284447,-0.048304,...,-0.038815,0.134425,-0.393854,0.585182,0.164889,0.223200,0.424681,0.759588,-0.548991,0.295145
4,한남동,-0.256816,-0.327690,0.112029,-0.076698,0.764227,-0.122812,0.052464,-0.115320,0.022177,...,-0.147396,0.506568,0.247860,0.377767,0.070646,0.816701,0.767942,0.039557,0.004635,0.999133
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
419,가회동,-0.285618,-0.238629,-0.206434,-0.441033,-0.344004,-0.001247,-0.035807,-0.139746,0.515529,...,-0.525358,0.001921,0.056336,0.459277,0.293834,-0.055779,-0.104890,0.790592,-0.502772,0.228831
420,종로1·2·3·4가동,-0.177895,-0.489362,-0.339834,-0.051969,-0.803621,0.064067,-0.224834,-0.126970,0.425846,...,-0.677298,0.283205,0.215208,0.069892,0.584104,-0.036702,-0.598967,0.230868,-0.694536,0.549405
421,방배2동,-0.045956,0.023942,-0.247413,0.458024,-0.011637,0.106036,0.019373,0.406274,0.020199,...,0.235108,0.077024,-0.090104,-0.050742,-0.249249,-0.110147,0.106479,0.023922,-0.062751,-0.136664
422,미아동,0.006720,0.289082,0.040417,0.684803,0.182243,0.192362,0.120329,0.392104,-0.069831,...,-0.195398,0.253809,0.348360,0.083346,-0.664251,-0.222044,0.448310,0.336182,-0.493172,-0.214582
